In [1]:
import os
import sys
from pathlib import Path
path = "C:\github/aide-analysis/"
sys.path.append(path)

In [2]:
from core.config import Config
config = Config()

In [3]:
# API KEY
vworld_key = config.OPEN_API['vworld']

###  Vworld Data API

In [4]:
import requests

def get_data(emdCd):
    """
    Vworld API 호출
    """
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"
    
    data = "LP_PA_CBND_BUBUN"
    
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"emdCd:like:{emdCd}"

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    return featureCollection

In [5]:
from src.utils import get_params_by_address

# 파라미터 설정
address_name = "백현동 530"
docs = get_params_by_address(address_name)

# 파라미터 파싱
pnuCode = docs['pnu_code']
sidoCd = pnuCode[:2]
emdCd = pnuCode[:8]
lon, lat = docs['lon'], docs['lat']

docs

{'pnu_code': '4113511000105300000',
 'address_name': '경기 성남시 분당구 백현동 530',
 'address_road_name': '경기 성남시 분당구 판교역로 145',
 'building_name': '알파리움2단지',
 'sido_name': '경기',
 'sigungu_name': '성남시 분당구',
 'road_name': '판교역로',
 'bdong_code': '4113511000',
 'bdong_name': '백현동',
 'hdong_code': '4113565700',
 'hdong_name': '백현동',
 'lon': '127.108437207085',
 'lat': '37.3935897511352'}

In [6]:
# GeoJson 데이터 호출
import json
geo = get_data(emdCd)
geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[127.09885646326357, 37.37822805470329],
       [127.0988140770859, 37.37824443403201],
       [127.09860546359174, 37.37832496718805],
       [127.09858087367508, 37.37833446607188],
       [127.09854238234905, 37.37881515423625],
       [127.09854346551867, 37.378814585706856],
       [127.09854445842126, 37.37881406230238],
       [127.0985471776185, 37.37881262745443],
       [127.09854955839835, 37.378811427148456],
       [127.09855193916655, 37.3788102178323],
       [127.0985540040872, 37.37880923402949],
       [127.09855608027515, 37.378808232197045],
       [127.09855644134325, 37.37880805169738],
       [127.09855693784091, 37.378807826035725],
       [127.09855735533894, 37.37880762746942],
       [127.09856171094606, 37.378805623637675],
       [127.09859062777231, 37.37878925545701],
       [127.0986294454792, 37.37877211327541],
       [127.09860

In [14]:
# from src.bigquery import bigquery_client, bigquery
# bq = bigquery_client
# sql = f"""
# SELECT T1.pnuCode, T1.contractYear, T1.contractMonth, T1.tradingSupplyUnitPriceAverage, T1.number
# FROM(
#     SELECT pnuCode 
#            ,contractYear
#            ,contractMonth, max(tradingSupplyUnitPriceAverage) AS tradingSupplyUnitPriceAverage
#            ,RANK() OVER(PARTITION BY pnuCode ORDER BY contractYear DESC, contractMonth DESC) AS number
#     FROM `aidepartners.etl.apartment_danji_monthly_price`
#     group by pnuCode, contractYear, contractMonth
#     order by pnuCode, contractYear DESC, contractMonth DESC
#     ) AS T1
# WHERE T1.tradingSupplyUnitPriceAverage is not null  AND MIN(T1.number)
# """
# #print(sql)
# dataframe = bq.query(sql).to_dataframe()
# dataframe

In [ ]:
import folium

# 줌 설정하기
zoom_size = 18

# folium 지도 생성하기
m = folium.Map(
    location=[lat, lon],
    zoom_start=zoom_size
)

m